# T1 and T2 Coherence Time Measurements

This notebook demonstrates comprehensive coherence time measurements in LeeQ.

## Contents
- T1 (relaxation time) measurements
- T2 (dephasing time) measurements
- Spin echo experiments
- Ramsey interferometry
- Data analysis and fitting techniques

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Core LeeQ imports
from leeq.chronicle import Chronicle, log_and_record
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
from leeq.experiments.experiments import ExperimentManager
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon

# Import coherence experiments using EPII v0.2.0 canonical names
from leeq.experiments.builtin.basic.characterizations import (
    T1Measurement,
    T2RamseyMeasurement,
    T2EchoMeasurement
)
from leeq.experiments.builtin.basic.calibrations import (
    RabiAmplitudeCalibration,
    MeasurementStatistics
)

# Start Chronicle logging
Chronicle().start_log()
log_and_record("coherence_measurements_start", {
    "notebook": "t1_t2_measurements",
    "focus": "comprehensive_coherence_analysis"
})

print("T1/T2 Coherence Measurements - LeeQ Example Notebook")
print("Using EPII v0.2.0 canonical experiment names")
print("Chronicle logging started")

# Setup simulation environment with realistic coherence parameters
manager = ExperimentManager()
manager.clear_setups()

# Create virtual transmon with well-defined coherence times
coherence_qubit = VirtualTransmon(
    name="CoherenceTestQubit",
    qubit_frequency=4987.5,  # MHz
    anharmonicity=-198,
    t1=65,  # Realistic T1
    t2=32,  # T2 < T1 due to dephasing 
    readout_frequency=9234.7,
    quiescent_state_distribution=np.asarray([0.89, 0.08, 0.025, 0.005])
)

# Setup high-level simulation
setup = HighLevelSimulationSetup(
    name='CoherenceTestSetup',
    virtual_qubits={1: coherence_qubit}
)
manager.register_setup(setup)

print(f"Coherence test system configured:")
print(f"  Qubit: {coherence_qubit.name}")
print(f"  Frequency: {coherence_qubit.qubit_frequency} MHz")
print(f"  Expected T1: {coherence_qubit.t1} μs")
print(f"  Expected T2: {coherence_qubit.t2} μs")
print(f"  T2/T1 ratio: {coherence_qubit.t2/coherence_qubit.t1:.2f}")

## T1 Relaxation Time Measurements

T1 measures the time for energy relaxation from |1⟩ to |0⟩. This is a fundamental limit for quantum gate operations.

### Theory
The T1 decay follows an exponential:
$$P_1(t) = P_0 \cdot e^{-t/T_1} + P_\infty$$

where P₀ is the initial excited probability and P∞ is the thermal equilibrium population.

### Key Points
- Measures energy loss to environment
- Sets upper bound for quantum gate fidelity  
- Typically ranges from 10-200 μs for superconducting qubits

In [ ]:
# First calibrate Rabi pulse
print("=" * 60)
print("T1 RELAXATION TIME MEASUREMENT")
print("=" * 60)

# Step 1: Calibrate π-pulse amplitude
print("Step 1: Calibrating π-pulse amplitude...")
rabi_cal = RabiAmplitudeCalibration(
    name="RabiForT1",
    qubit=1,
    drive_frequency=coherence_qubit.qubit_frequency,
    amplitude_start=0.0,
    amplitude_stop=1.0,
    amplitude_points=31,
    pulse_width=0.05,
    repeated_measurement_count=800
)

rabi_results = rabi_cal.run()
amplitudes = rabi_results['sweep_values'] 
probabilities = rabi_results['measurement_probabilities']
pi_amplitude = amplitudes[np.argmax(probabilities)]

print(f"✓ Calibrated π-pulse amplitude: {pi_amplitude:.4f}")

# Step 2: T1 measurement
print("\nStep 2: Running T1 measurement...")
t1_experiment = T1Measurement(
    name="T1_Characterization",
    qubit=1,
    delay_start=0.0,
    delay_stop=200.0,  # 200 μs range
    delay_points=41,   # Good resolution
    pi_pulse_amplitude=pi_amplitude,
    repeated_measurement_count=1200
)

t1_results = t1_experiment.run()
t1_delays = t1_results['sweep_values']
t1_probabilities = t1_results['measurement_probabilities']

print(f"✓ T1 measurement completed with {len(t1_delays)} points")

# Step 3: Fit exponential decay
def exponential_decay(t, amplitude, t1, offset):
    """Exponential decay model for T1."""
    return amplitude * np.exp(-t / t1) + offset

# Fit T1 decay
try:
    # Initial guess: [amplitude, T1, offset]
    p0 = [0.8, 65, 0.1]  
    popt, pcov = curve_fit(exponential_decay, t1_delays, t1_probabilities, p0=p0)
    
    fitted_amplitude = popt[0]
    fitted_t1 = popt[1] 
    fitted_offset = popt[2]
    
    # Calculate uncertainties
    param_errors = np.sqrt(np.diag(pcov))
    t1_error = param_errors[1]
    
    fit_success = True
    
    print(f"\n✓ T1 Fit Results:")
    print(f"  T1 = {fitted_t1:.1f} ± {t1_error:.1f} μs")
    print(f"  Initial amplitude = {fitted_amplitude:.3f}")
    print(f"  Thermal offset = {fitted_offset:.3f}")
    print(f"  Expected T1 = {coherence_qubit.t1} μs")
    
except Exception as e:
    print(f"✗ T1 fit failed: {e}")
    fitted_t1 = None
    fit_success = False

# Visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('T1 Decay Curve', 'T1 Fit Quality'),
    column_widths=[0.7, 0.3]
)

# T1 decay plot
fig.add_trace(
    go.Scatter(
        x=t1_delays,
        y=t1_probabilities,
        mode='markers',
        name='T1 Data',
        marker=dict(size=6, color='blue')
    ),
    row=1, col=1
)

if fit_success:
    # Add fit curve
    t_fit = np.linspace(0, max(t1_delays), 200)
    p_fit = exponential_decay(t_fit, *popt)
    
    fig.add_trace(
        go.Scatter(
            x=t_fit,
            y=p_fit,
            mode='lines',
            name=f'Exp Fit (T1={fitted_t1:.1f}μs)',
            line=dict(color='red', width=2)
        ),
        row=1, col=1
    )
    
    # Add 1/e line
    fig.add_hline(
        y=fitted_offset + fitted_amplitude/np.e,
        line_dash="dash",
        line_color="gray",
        annotation_text="1/e point",
        row=1, col=1
    )
    
    # Residuals plot
    residuals = t1_probabilities - exponential_decay(t1_delays, *popt)
    fig.add_trace(
        go.Scatter(
            x=t1_delays,
            y=residuals,
            mode='markers',
            name='Residuals',
            marker=dict(size=4, color='green')
        ),
        row=1, col=2
    )
    
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=2)

fig.update_xaxes(title_text="Delay (μs)", row=1, col=1)
fig.update_yaxes(title_text="Excited Probability", row=1, col=1)
fig.update_xaxes(title_text="Delay (μs)", row=1, col=2)
fig.update_yaxes(title_text="Residuals", row=1, col=2)

fig.update_layout(
    height=500,
    title_text="T1 Relaxation Time Analysis",
    showlegend=True
)

fig.show()

# Log T1 results
log_and_record("t1_measurement_results", {
    "fitted_t1_us": fitted_t1,
    "expected_t1_us": coherence_qubit.t1,
    "fit_success": fit_success,
    "pi_pulse_amplitude": pi_amplitude,
    "delays_us": t1_delays.tolist(),
    "probabilities": t1_probabilities.tolist()
})

## T2 Dephasing Time Measurements

T2 measures coherence loss due to phase randomization. Two main types:
- **T2* (Ramsey)**: Includes low-frequency noise effects
- **T2 (Echo)**: Removes slow noise via spin echo

### Theory
**Ramsey**: $P_1(t) = A \cdot e^{-t/T_2^*} \cos(2\pi f t + \phi) + B$
**Echo**: Similar decay without oscillations

### Key Points
- T2 ≤ 2×T1 (fundamental limit)
- T2* ≤ T2 due to low-frequency noise
- Critical for superposition state lifetimes

In [ ]:
# T2 Ramsey and Echo measurements
print("\n" + "=" * 60)
print("T2 DEPHASING TIME MEASUREMENTS")
print("=" * 60)

# Step 1: T2* Ramsey measurement  
print("Step 1: T2* Ramsey measurement...")
t2_ramsey = T2RamseyMeasurement(
    name="T2_Ramsey_Measurement",
    qubit=1,
    delay_start=0.0,
    delay_stop=120.0,  # Longer than T2
    delay_points=31,
    pi_half_pulse_amplitude=pi_amplitude/2,
    detuning=0.2,  # 200 kHz detuning for clear fringes
    repeated_measurement_count=1000
)

ramsey_results = t2_ramsey.run()
ramsey_delays = ramsey_results['sweep_values']
ramsey_probabilities = ramsey_results['measurement_probabilities']

print(f"✓ Ramsey measurement completed")

# Fit Ramsey oscillations
def ramsey_model(t, amplitude, t2_star, frequency, phase, offset):
    """Damped oscillation model for Ramsey."""
    return amplitude * np.exp(-t / t2_star) * np.cos(2 * np.pi * frequency * t + phase) + offset

try:
    # Initial guess: [amplitude, T2*, frequency, phase, offset]
    p0 = [0.4, 32, 0.2, 0, 0.5]
    popt_ramsey, _ = curve_fit(ramsey_model, ramsey_delays, ramsey_probabilities, p0=p0)
    
    fitted_t2_star = popt_ramsey[1]
    fitted_freq = popt_ramsey[2]
    
    print(f"✓ Ramsey Fit Results:")
    print(f"  T2* = {fitted_t2_star:.1f} μs")
    print(f"  Fitted frequency = {fitted_freq:.3f} MHz")
    print(f"  Expected detuning = {t2_ramsey.detuning:.3f} MHz")
    print(f"  Expected T2 = {coherence_qubit.t2} μs")
    
    ramsey_fit_success = True
    
except Exception as e:
    print(f"✗ Ramsey fit failed: {e}")
    fitted_t2_star = None
    ramsey_fit_success = False

# Step 2: T2 Echo measurement (if implemented)
print("\nStep 2: T2 Echo measurement...")
try:
    t2_echo = T2EchoMeasurement(
        name="T2_Echo_Measurement", 
        qubit=1,
        delay_start=0.0,
        delay_stop=150.0,
        delay_points=31,
        pi_pulse_amplitude=pi_amplitude,
        pi_half_pulse_amplitude=pi_amplitude/2,
        repeated_measurement_count=1000
    )
    
    echo_results = t2_echo.run()
    echo_delays = echo_results['sweep_values']
    echo_probabilities = echo_results['measurement_probabilities']
    
    # Fit echo decay (exponential without oscillations)
    def echo_decay(t, amplitude, t2_echo, offset):
        return amplitude * np.exp(-t / t2_echo) + offset
        
    popt_echo, _ = curve_fit(echo_decay, echo_delays, echo_probabilities, 
                            p0=[0.4, 50, 0.5])
    fitted_t2_echo = popt_echo[1]
    
    print(f"✓ Echo Fit Results:")
    print(f"  T2 (echo) = {fitted_t2_echo:.1f} μs")
    
    echo_fit_success = True
    
except Exception as e:
    print(f"Note: T2 Echo measurement not available or failed: {e}")
    echo_fit_success = False
    fitted_t2_echo = None

# Comprehensive visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('T2* Ramsey Oscillations', 'Ramsey Envelope', 'T2 Echo Decay', 'Coherence Summary'),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'bar'}]]
)

# Ramsey plot
fig.add_trace(
    go.Scatter(
        x=ramsey_delays,
        y=ramsey_probabilities,
        mode='markers',
        name='Ramsey Data',
        marker=dict(size=5, color='blue')
    ),
    row=1, col=1
)

if ramsey_fit_success:
    # Add fit
    t_fit = np.linspace(0, max(ramsey_delays), 200)
    p_fit = ramsey_model(t_fit, *popt_ramsey)
    
    fig.add_trace(
        go.Scatter(
            x=t_fit,
            y=p_fit,
            mode='lines',
            name=f'Ramsey Fit (T2*={fitted_t2_star:.1f}μs)',
            line=dict(color='red', width=2)
        ),
        row=1, col=1
    )
    
    # Envelope
    envelope = popt_ramsey[0] * np.exp(-t_fit / popt_ramsey[1]) + popt_ramsey[4]
    fig.add_trace(
        go.Scatter(
            x=t_fit,
            y=envelope,
            mode='lines',
            name='T2* Envelope',
            line=dict(color='green', dash='dash', width=2)
        ),
        row=1, col=2
    )
    
    # Lower envelope
    lower_envelope = 2*popt_ramsey[4] - envelope
    fig.add_trace(
        go.Scatter(
            x=t_fit,
            y=lower_envelope,
            mode='lines',
            name='Lower Envelope',
            line=dict(color='green', dash='dash', width=2),
            showlegend=False
        ),
        row=1, col=2
    )

# Echo plot (if available)
if echo_fit_success:
    fig.add_trace(
        go.Scatter(
            x=echo_delays,
            y=echo_probabilities,
            mode='markers',
            name='Echo Data', 
            marker=dict(size=5, color='orange')
        ),
        row=2, col=1
    )
    
    t_echo_fit = np.linspace(0, max(echo_delays), 200)
    p_echo_fit = echo_decay(t_echo_fit, *popt_echo)
    
    fig.add_trace(
        go.Scatter(
            x=t_echo_fit,
            y=p_echo_fit,
            mode='lines',
            name=f'Echo Fit (T2={fitted_t2_echo:.1f}μs)',
            line=dict(color='purple', width=2)
        ),
        row=2, col=1
    )

# Summary bar chart
coherence_times = []
labels = []
colors = []

if fit_success and fitted_t1 is not None:
    coherence_times.append(fitted_t1)
    labels.append('T1')
    colors.append('blue')
    
if ramsey_fit_success and fitted_t2_star is not None:
    coherence_times.append(fitted_t2_star)
    labels.append('T2*')
    colors.append('green')
    
if echo_fit_success and fitted_t2_echo is not None:
    coherence_times.append(fitted_t2_echo)
    labels.append('T2 (echo)')
    colors.append('orange')

if coherence_times:
    fig.add_trace(
        go.Bar(
            x=labels,
            y=coherence_times,
            name='Measured Times',
            marker_color=colors
        ),
        row=2, col=2
    )
    
    # Add expected values as lines
    fig.add_hline(y=coherence_qubit.t1, line_dash="dash", line_color="blue", 
                  annotation_text=f"Expected T1={coherence_qubit.t1}μs", row=2, col=2)
    fig.add_hline(y=coherence_qubit.t2, line_dash="dash", line_color="green",
                  annotation_text=f"Expected T2={coherence_qubit.t2}μs", row=2, col=2)

# Update layout
fig.update_xaxes(title_text="Delay (μs)", row=1, col=1)
fig.update_yaxes(title_text="Excited Probability", row=1, col=1)
fig.update_xaxes(title_text="Delay (μs)", row=1, col=2)  
fig.update_yaxes(title_text="Excited Probability", row=1, col=2)
fig.update_xaxes(title_text="Delay (μs)", row=2, col=1)
fig.update_yaxes(title_text="Excited Probability", row=2, col=1)
fig.update_xaxes(title_text="Coherence Time", row=2, col=2)
fig.update_yaxes(title_text="Time (μs)", row=2, col=2)

fig.update_layout(
    height=800,
    title_text="Comprehensive T1/T2 Coherence Analysis",
    showlegend=True
)

fig.show()

# Final summary
print("\n" + "=" * 60)
print("COHERENCE MEASUREMENT SUMMARY")
print("=" * 60)

if fit_success and fitted_t1:
    print(f"T1 (relaxation): {fitted_t1:.1f} μs")
if ramsey_fit_success and fitted_t2_star:
    print(f"T2* (ramsey): {fitted_t2_star:.1f} μs")
if echo_fit_success and fitted_t2_echo:
    print(f"T2 (echo): {fitted_t2_echo:.1f} μs")

print(f"\nExpected values:")
print(f"T1: {coherence_qubit.t1} μs")
print(f"T2: {coherence_qubit.t2} μs")

# Log comprehensive results
log_and_record("coherence_measurement_summary", {
    "fitted_t1_us": fitted_t1,
    "fitted_t2_star_us": fitted_t2_star,
    "fitted_t2_echo_us": fitted_t2_echo,
    "expected_t1_us": coherence_qubit.t1,
    "expected_t2_us": coherence_qubit.t2,
    "measurement_success": {
        "t1": fit_success,
        "t2_ramsey": ramsey_fit_success,
        "t2_echo": echo_fit_success
    }
})

print("\nCoherence measurements completed and logged to Chronicle!")